In [1]:
#This builds a dataset using new criteria:
#Mutations at the same coordinates are treated as one
#CIMP low is considered its own class: 2.0
#There exists a control set! for these samples, place them in their own dataset
import os
import csv

#import data of coad and read
clinfile = "../../crcclasses.csv"
clinfile2 = "../../crcclasses2.csv"
coadmutextfile = "../../COADsomatic.csv"
readmutextfile = "../../READsomatic.csv"
with open(coadmutextfile,'r') as f:
        it = csv.reader(f)
        coadmutextall = list(it)
with open(readmutextfile,'r') as f:
        it = csv.reader(f)
        readmutextall = list(it)
with open(clinfile,'r') as f:
        it = csv.reader(f)
        clinall = list(it)
with open(clinfile2,'r') as f:
        it = csv.reader(f)
        clinall2 = list(it)
        
print("finished loading")

finished loading


In [2]:
# build dictionary with classifications
myclass = ''
foreback = {}
for row in clinall:
    if(row[0] == ""):
        continue
    if(row[2] == 'Normal'):
        myclass = 'c'
    elif(row[2] == 'CIMP+'):
        myclass = '1.0'
    elif(row[2] == 'CIMPi'):
        myclass = '2.0'
    elif(row[2] == 'CIMP-'):
        myclass = '-1.0'
    else:
        print("alert")
    name = row[0].split(".")
    foreback[name[0]+'-'+name[1]+'-'+name[2]+'-'+name[3]] = myclass
for row in clinall2:
    if(row[0] == ""):
        continue
    if(row[2] == 'control'):
        myclass = 'c'
    elif(row[2] == 'CIMP+'):
        myclass = '1.0'
    elif(row[2] == 'CIMPi'):
        myclass = '2.0'
    elif(row[2] == 'CIMP-'):
        myclass = '-1.0'
    else:
        print("alert")
    name = row[0].split("-")
    foreback[name[0]+'-'+name[1]+'-'+name[2]+'-'+name[3]] = myclass

del clinall


### parse through mutations and data_clinical to store unique mutations and their associated samples

In [3]:
presents = {} #map samp to all present mutations
presents_control = {}
realrow = [""] #preprocessing needs to be done so only mutations with more than one occurrence is shown
firstrow = [""] #this will list all mutations at the top
firstrow_control = [""]
discardmuts = []
discardsamps = []
notinforeback = []
#coad first
#find mutations within various samples, cut off past
for x in range(1,len(coadmutextall)):
    #only use the "mutation" if it's labelled somatic...
    if(coadmutextall[x][10] != "Somatic"):
        continue
    pieces = coadmutextall[x][9].split("-")
    inp = pieces[0] + "-" + pieces[1] + "-" + pieces[2] + '-' + pieces[3][:-1]
    newmut = (coadmutextall[x][0] +"_GRCh38_"+ coadmutextall[x][1][3:] + ":" +coadmutextall[x][2] + "-"+ coadmutextall[x][3] + "_" + coadmutextall[x][4] +"_"+ coadmutextall[x][5] + "_" + coadmutextall[x][6] + "_" + coadmutextall[x][7] + "_" + coadmutextall[x][8])
    if(inp not in foreback):
        if(inp not in notinforeback):
            notinforeback.append(inp)
        if(inp not in discardsamps):
            discardsamps.append(inp)
        if(newmut not in discardmuts):
            discardmuts.append(newmut)
        continue
    #if we get to this point, the sample is valid
    #we next need to check if it is a control. if so, it goes into its own data set
    if(foreback[inp] == 'c'):
        if newmut not in firstrow_control:
            firstrow_control.append(newmut)
        if inp in presents_control:
            presents_control[inp].append(newmut)
        else:
            presents_control[inp] = [newmut]
        continue
    #if we get to this point, the sample is not control
    if newmut not in firstrow:
        firstrow.append(newmut)
    else:
        realrow.append(newmut) #it has occurred at least twice
    if inp in presents:
        presents[inp].append(newmut)
    else:
        presents[inp] = [newmut]
#now do the same thing with read        
for x in range(1,len(readmutextall)):
    #only use the "mutation" if it's labelled somatic...
    if(readmutextall[x][10] != "Somatic"):
        continue
    pieces = readmutextall[x][9].split("-")
    newmut = (readmutextall[x][0] +"_GRCh38_"+ readmutextall[x][1][3:] + ":" +readmutextall[x][2] + "-"+ readmutextall[x][3] + "_" + readmutextall[x][4] +"_"+ readmutextall[x][5] + "_" + readmutextall[x][6] + "_" + readmutextall[x][7] + "_" + readmutextall[x][8])
    inp = pieces[0] + "-" + pieces[1] + "-" + pieces[2] + '-' + pieces[3][:-1]
    if(inp not in foreback):
        if(inp not in notinforeback):
            notinforeback.append(inp)
        if(inp not in discardsamps):
            discardsamps.append(inp)
        if(newmut not in discardmuts):
            discardmuts.append(newmut)
        continue
    #we next need to check if it is a control. if so, it goes into its own data set
    if(foreback[inp] == 'c'):
        if newmut not in firstrow_control:
            firstrow_control.append(newmut)
        if inp in presents_control:
            presents_control[inp].append(newmut)
        else:
            presents_control[inp] = [newmut]
        continue
    #if we get to this point, the sample is not control
    if newmut not in firstrow:
        firstrow.append(newmut)
    else:
        realrow.append(newmut)
    if inp in presents:
        presents[inp].append(newmut)
    else:
        presents[inp] = [newmut]
realrow.append("class")
firstrow.append("class")
firstrow_control.append("class")
firstrow_control

['', 'class']

In [4]:
numfeats = len(realrow) - 2
pruned = len(firstrow) - len(realrow)
numfeats_control = len(firstrow_control) - 2

print("num features: ",str(numfeats))
print("num features in control set: ",str(numfeats_control))
#find foreground vs. background

#newset = [firstrow]
newset = [realrow]
newset_control = [firstrow_control]

#attach to new matrix
for key,muts in presents.items():
    nextrow = [key]
    #for item in firstrow:
    for item in realrow:
        if(item != "" and item != "class"):
            if item in muts:
                nextrow.append("1.0")
            else:
                nextrow.append("0.0")
    nextrow.append(foreback[key])
    newset.append(nextrow)

#do same for control group    
for key,muts in presents_control.items():
    nextrow = [key]
    for item in firstrow_control:
        if(item != "" and item != "class"):
            if item in muts:
                nextrow.append("1.0")
            else:
                nextrow.append("0.0")
    nextrow.append(foreback[key])
    newset_control.append(nextrow)
    
with open("../../PANCANcoadread.csv","w") as fun:
    writer = csv.writer(fun)
    writer.writerows(newset)
with open("../../PANCANcoadread_control.csv","w") as fun:
    writer = csv.writer(fun)
    writer.writerows(newset_control)

num features:  15747
num features in control set:  0


In [5]:
hcount = 0
lcount = 0
ncount = 0
ccount = 0
for row in newset:
    if(row[0] == ""):
        continue
    if(row[len(newset[0])-1] == 'c'):
        ccount += 1
    elif(float(row[len(newset[0])-1]) == 1.0):
        hcount += 1
    elif(float(row[len(newset[0])-1]) == 2.0):
        lcount += 1
    elif(float(row[len(newset[0])-1]) == -1.0):
        ncount += 1
print("high: ",hcount," low: ",lcount," neg: ",ncount, " control: ",ccount)
print(len(newset))

high:  143  low:  182  neg:  175  control:  0
501


In [6]:
len(discardsamps)

35

In [7]:
newdiscardmuts = [item for item in discardmuts if item not in newset[0]]
len(newdiscardmuts)

15208

In [8]:
len(newset_control)

1

In [9]:
len(newset[0])-2

15747

In [10]:
len(newset_control[0])-2

0

In [11]:
print("Number of features with only one occurrence: ",pruned)

Number of features with only one occurrence:  253933


In [12]:
len(foreback)

4959

In [13]:
notinforeback

['TCGA-5M-AAT5-01',
 'TCGA-AY-A71X-01',
 'TCGA-WS-AB45-01',
 'TCGA-T9-A92H-01',
 'TCGA-AY-A69D-01',
 'TCGA-RU-A8FL-01',
 'TCGA-5M-AATA-01',
 'TCGA-4N-A93T-01',
 'TCGA-AA-3841-01',
 'TCGA-QL-A97D-01',
 'TCGA-NH-A5IV-01',
 'TCGA-NH-A6GA-01',
 'TCGA-NH-A6GC-01',
 'TCGA-5M-AAT4-01',
 'TCGA-AA-3675-01',
 'TCGA-3L-AA1B-01',
 'TCGA-NH-A50U-01',
 'TCGA-NH-A6GB-01',
 'TCGA-5M-AATE-01',
 'TCGA-SS-A7HO-01',
 'TCGA-NH-A8F7-01',
 'TCGA-NH-A8F8-01',
 'TCGA-5M-AAT6-01',
 'TCGA-AY-A8YK-01',
 'TCGA-AA-3967-01',
 'TCGA-4T-AA8H-01',
 'TCGA-AG-3906-01',
 'TCGA-AF-5654-01',
 'TCGA-AG-4009-01',
 'TCGA-AF-A56K-01',
 'TCGA-AF-3914-01',
 'TCGA-AG-3728-01',
 'TCGA-AF-A56N-01',
 'TCGA-AG-3891-01',
 'TCGA-AF-A56L-01']

In [14]:
len(discardmuts)

15812